In [ ]:
"""mc_high_dimensions.ipynb"""
# Cell 1

from __future__ import annotations

import typing

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import MarkerStyle
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from numba import float64, vectorize  # type: ignore
from scipy.signal import find_peaks  # type: ignore
from scipy.special import gamma  # type: ignore

if typing.TYPE_CHECKING:
    from matplotlib.axes import Axes
    from numpy.typing import NDArray

%matplotlib widget


@vectorize([float64(float64, float64)], nopython=True)  # type: ignore
def halton(n: float, p: int) -> float:
    h = 0
    f = 1
    while n > 0:
        f: float = f / p
        h += (n % p) * f
        n = int(n / p)
    return h


def plot_actual(ax: Axes) -> None:
    act_x: NDArray[np.float_] = np.linspace(0, 12, 1000)
    act_y: NDArray[np.float_] = np.power(np.pi, act_x / 2) / gamma(act_x / 2 + 1)
    ax.plot(act_x, act_y, color="red", label="Actual", linewidth=2)
    max_idx: int = int(find_peaks(act_y)[0])  # type: ignore
    max_dim: float = act_x[max_idx]
    max_vol: float = act_y[max_idx]
    ax.scatter(max_dim, max_vol, marker=MarkerStyle("o"), color="green")
    ax.vlines(max_dim, 0, max_vol, color="green")
    ax.axhline(max_vol, linestyle="dashed", color="darkgray")

    # fmt: off
    ax.text(5.5, 1.5,
        f"Maximum Dimension: {max_dim:.5f}\nMaximum Volume: {max_vol:.5f}", fontsize="large",
        bbox=dict(boxstyle="square", facecolor="white", edgecolor="green"))
    # fmt: on


def plot_hypersphere_volume(ax: Axes) -> None:
    iterations: int = 50**4
    print(f"Dots per dimension = {iterations:,}")

    primes: list[int] = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]

    dimensions = 13

    est_volume: NDArray[np.float_] = np.zeros(dimensions)
    est_volume[0] = 1  # By definition
    est_volume[1] = 2  # The 1-D interval [-1,1] has length 2

    # Prime the loop by generating the X-coordinates for the 2-D circle
    v: float = halton(np.arange(iterations), primes[0]) * 2 - 1  # type: ignore
    d: float = v**2

    for dimension in range(2, dimensions):
        print(f"Calculating the volume of a unit {dimension:>2}-ball . . .")
        v = halton(np.arange(iterations), primes[dimension - 1]) * 2 - 1  # type: ignore
        d += v**2
        est_volume[dimension] = (
            np.count_nonzero(d <= 1.0) / iterations * np.power(2, dimension)
        )

    ax.plot(
        np.arange(dimensions), est_volume, color="blue", label="Estimated", linewidth=2
    )

    plot_actual(ax)

    ax.set_title("Volume of n-Dimensional Hyperspheres")
    ax.set_xlabel("Dimension")
    ax.set_ylabel("Volume")
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.xaxis.set_minor_locator(MultipleLocator(0.5))
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.grid()
    ax.legend()


def main() -> None:
    plt.close("all")
    plt.figure(" ", figsize=(12, 8))
    plot_hypersphere_volume(plt.axes())
    plt.show()


main()